# SCRAPING DEI PERCORSI FERROVIARI DA KAYAK

In [2]:
#install packages for web scraping:Selenium (emulate a user on a web site) + chromium
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 904 kB 13.4 MB/s 
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Rel

In [3]:
import sys #to setup the path of chromedriver
import logging
from selenium import webdriver
from selenium.webdriver.remote.remote_connection import LOGGER
LOGGER.setLevel(logging.WARNING)
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import json #to work with json file
import pprint #to print stuff in a more aesthetic way

In [4]:
#other stuff to work in a browser
chrome_options = webdriver.ChromeOptions() #With webdriver we can start the phantom browser, emulate the user navigation and scrape our data.
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

# START SCRAPING

In [5]:
wd = webdriver.Chrome('chromedriver',options=chrome_options) #è un oggetto che mi permette di interfacciarmi con web tramite richieste

In [6]:
def parse_train(train,railway):  
    departure_station = ""
    arrival_station = ""
    date = ""
    departure_hour =""
    return_hour = ""
    change = ""
    price = ""
    type_of_train = ""

    try:
      if (len(wd.find_elements_by_css_selector('div[id*="inner"]> div > div.col-info.result-column > div.top-row > div > div > div.bfLabel.bf-transportation')) > 0):
        pass
      if (len(train.find_elements_by_css_selector('div[id*="info-leg-0-origin-airport"]')) > 0):
          departure_station = train.find_elements_by_css_selector('div[id*="info-leg-0-origin-airport"]')[0].text
      if (len(train.find_elements_by_css_selector('div[id*="info-leg-0-destination-airport"]')) > 0):
          arrival_station = train.find_elements_by_css_selector('div[id*="info-leg-0-destination-airport"]')[0].text   
      if (len(train.find_elements_by_css_selector("span.date-text")) > 0):
          date = train.find_elements_by_css_selector("span.date-text")[0].text       
      if (len(train.find_elements_by_css_selector('div[id*="info-leg-0"]> div > div.section.times > div.top > span:nth-child(1) > span')) > 0):
          departure_hour = train.find_elements_by_css_selector('div[id*="info-leg-0"]> div > div.section.times > div.top > span:nth-child(1) > span')[0].text
      if (len(train.find_elements_by_css_selector('div[id*="info-leg-0"]> div > div.section.times > div.top > span:nth-child(3) > span')) > 0):
          return_hour = train.find_elements_by_css_selector('div[id*="info-leg-0"]> div > div.section.times > div.top > span:nth-child(3) > span')[0].text 
      if (len(train.find_elements_by_css_selector('div[id*="info-leg-0"]> div > div.section.stops > div.top > span')) > 0):
          change = train.find_elements_by_css_selector('div[id*="info-leg-0"]> div > div.section.stops > div.top > span')[0].text       
      if (len(train.find_elements_by_css_selector('span[id*="price-text"]')) > 0):
          price = train.find_elements_by_css_selector('span[id*="price-text"]')[0].text    
      if (len(train.find_elements_by_css_selector('div[id*="extra-info"]> div > div > span')) > 0):
          type_of_train = train.find_elements_by_css_selector('div[id*="extra-info"]> div > div > span')[0].text 
    except:
      pass
    return {'railway': railway,
            'departure station': departure_station,
            'arrival station': arrival_station, 
            'date' : departure_date,
            'departure time': departure_hour,
            'arrival time': return_hour,
            'change': change,
            'price': price,
            'type of train': type_of_train
            }

In [86]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from tqdm import tqdm
import numpy as np

train_details = []
train_details_raw = []

#elenco delle tratte ferroviarie
# list_railways = ['MIL-TRN','MIL-ROM','MIL-PAR','MIL-GVA','MIL-VIE','MIL-ZRH','MIL-PRG','MIL-NAP','MUC-STR','GVA-PAR','BRU-PAR','BRU-AMS','BRU-HAJ','AMS-PAR','BER-HAJ','BER-PAR','ZRH-VIE','PRG-MUC','BUD-PRG','VIE-MUC','ZAG-VIE','PAR-NCE','ROM-PAR','ROM-GVA','ROM-VIE','ROM-ZRH','ROM-MUC','ROM-TRN','FRA-PAR','FRA-AMS','FRA-MUC','FRA-PRG','BER-MUC','PRG-VIE','WAW-PRG','WAW-BER','AMS-HAM','HAM-BER','HAM-BRU','LON-PAR','LON-BRU','LON-AMS','MAD-VLC','PAR-LYS','TRN-LYS','MIL-LYS','BLQ-ROM','TRN-PAR','LYS-NCE','MIL-NCE','TRN-NCE','BCN-MAD','BCN-VLC','LON-MAN','MAD-SVQ','AMS-DUS','LON-EDI','ROM-VCE','BOL-NAP','ROM-NAP','TRN-MIL','ROM-MIL','PAR-MIL','GVA-MIL','VIE-MIL','ZRH-MIL','PRG-MIL','NAP-MIL','STR-MUC','PAR-GVA','PAR-BRU','AMS-BRU','HAJ-BRU','PAR-AMS','HAJ-BER','PAR-BER','VIE-ZRH','MUC-PRG','PRG-BUD','MUC-VIE','VIE-ZAG','NCE-PAR','PAR-ROM','GVA-ROM','VIE-ROM','ZRH-ROM','MUC-ROM','TRN-ROM','PAR-FRA','AMS-FRA','MUC-FRA','PRG-FRA','MUC-BER','VIE-PRG','PRG-WAW','BER-WAW','HAM-AMS','BER-HAM','BRU-HAM','PAR-LON','BRU-LON','AMS-LON','VLC-MAD','LYS-PAR','LYS-TRN','LYS-MIL','ROM-BLQ','PAR-TRN','NCE-LYS','NCE-MIL','NCE-TRN','MAD-BCN','VLC-BCN','MAN-LON','SVQ-MAD','DUS-AMS','EDI-LON','VCE-ROM','NAP-BOL','NAP-ROM''VIE-ZAG','VIE-STR','VIE-WAW','VIE-BEG','VIE-BUD','VIE-PRG','BRU-LIL','BRU-DUS','BRU-FRA','ZAG-MUC','ZAG-VCE','ZAG-BUD','BOD-LYS','BOD-NCE','BOD-TLS','BOD-GVA','LIL-PAR','LIL-AMS','LIL-DUS','LIL-LON','LYS-GVA','LYS-TLS','LYS-ZRH','NCE-GVA','TLS-BCN','HAM-DUS','HAM-HAJ','BER-PRG','BER-FRA','FRA-STR','FRA-ZRH','FRA-DUS','DUS-HAJ','MUC-ZRH','MUC-FRA','MUC-MIL','MUC-VCE','STR-ZRH','STR-DUS','BOL-MIL','BOL-VCE','WAW-BUD','VLC-SVQ','ZAG-VIE','STR-VIE','WAW-VIE','BEG-VIE','BUD-VIE','PRG-VIE','LIL-BRU','DUS-BRU','FRA-BRU','MUC-ZAG','VCE-ZAG','BUD-ZAG','LYS-BOD','NCE-BOD','TLS-BOD','GVA-BOD','PAR-LIL','AMS-LIL','DUS-LIL','LON-LIL','GVA-LYS','TLS-LYS','ZRH-LYS','GVA-NCE','BCN-TLS','DUS-HAM','HAJ-HAM','PRG-BER','FRA-BER','STR-FRA','ZRH-FRA','DUS-FRA','HAJ-DUS','ZRH-MUC','FRA-MUC','MIL-MUC','VCE-MUC','ZRH-STR','DUS-STR','MIL-BOL','VCE-BOL','BUD-WAW','SVQ-VLC']
list_railways = ['MIL-ROM']
departure_date = '2021-10-06'

In [87]:
len(list_railways)

1

In [88]:
# elenco dei viaggi diretti o con 1 cambio
for railway in tqdm(list_railways):
  wd.get(f"https://www.kayak.it/flights/{railway}/{departure_date}")
  time.sleep(5) #inserisco una pausa di 5 secondi dopo essere andato sul sito, per aspettare il caricamento

  if (len(wd.find_elements_by_css_selector("button.Common-Widgets-Button-StyleJamButton.ui-button.size-m._id7._ihr._ihs._h-Y._im4._ii0._ihp._ihq._inw._iir._iae._iam._ifT._ifS._jV7._iiN._3-._4M._40._k-A._isy._jUK._30._43")) > 0):#se ci sono i cookie da accettare, accettali
      print("Click cookie law button")
      wd.find_elements_by_css_selector("button.Common-Widgets-Button-StyleJamButton.ui-button.size-m._id7._ihr._ihs._h-Y._im4._ii0._ihp._ihq._inw._iir._iae._iam._ifT._ifS._jV7._iiN._3-._4M._40._k-A._isy._jUK._30._43")[0].click()
  if (len(wd.find_elements_by_id("onetrust-accept-btn-handler")) > 0):#se ci sono i cookie da accettare, accettali
        print("Click cookie law button")
        wd.find_elements_by_id("onetrust-accept-btn-handler")[0].click()
  wd.save_screenshot("check_browser.png") #verifico che i cookies sono accettati


  for i in range(0,2):
    try:
      WebDriverWait(wd, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ('a[class*="moreButton"]')))).click()
    except WebDriverException:
      print("Element is not clickable")
  list_train = wd.find_elements_by_css_selector('div[class*="resultInner"]')
  for train in tqdm(list_train):
    train_details.append(parse_train(train,railway))
    train_details_raw.append(train)
    break
  print("Click next page")
  time.sleep(5)
  
  list_train = wd.find_elements_by_css_selector('div[class*="resultInner"]')
  for train in tqdm(list_train):
    train_details.append(parse_train(train,railway))
    train_details_raw.append(train)
print(len(train_details))

  0%|          | 0/30 [00:00<?, ?it/s]


Click next page



100%|██████████| 1/1 [00:24<00:00, 24.27s/it]

46


In [89]:
import pandas as pd
ds_train_details1= pd.DataFrame(train_details)
ds_train_details_raw= pd.DataFrame(train_details_raw)
ds_train_details1.head()

,railway,departure station,arrival station,date,departure time,arrival time,change,price,type of train
0,MIL-ROM,MIL San Donato,ROM Tiburtina,2021-10-06,23:15,6:25,0 cambi,10 €,Itabus
1,MIL-ROM,MIL San Donato,ROM Tiburtina,2021-10-06,23:15,6:25,0 cambi,10 €,Itabus
2,MIL-ROM,MIL Centrale,ROM Tiburtina,2021-10-06,14:48,21:22,0 cambi,23 €,Trenitalia
3,MIL-ROM,MXP Aeroporto Malpensa,FCO Roma-Fiumicino,2021-10-06,16:05,21:35,1 scalo,16 €,Ryanair
4,MIL-ROM,MXP Aeroporto Malpensa,FCO Roma-Fiumicino,2021-10-06,14:20,18:45,1 scalo,17 €,"Wizz Air, Ryanair"


In [90]:
ds_train_details1.to_csv("ds_flight_details_sc1.csv", index=False)
#ds_train_details_raw.to_csv("ds_flight_details_raw_sc.csv", index=False)

# ETL

In [91]:
import pandas as pd
train_details = pd.read_csv('ds_flight_details_sc1.csv')
train_details.tail()

,railway,departure station,arrival station,date,departure time,arrival time,change,price,type of train
41,MIL-ROM,LIN Aeroporto Milano Linate,FCO Roma-Fiumicino,2021-10-06,18:40,22:05,1 scalo,332 €,Lufthansa
42,MIL-ROM,MXP Aeroporto Malpensa,FCO Roma-Fiumicino,2021-10-06,13:30,17:15,1 scalo,332 €,Lufthansa
43,MIL-ROM,MXP Aeroporto Malpensa,FCO Roma-Fiumicino,2021-10-06,6:50,12:25,1 scalo,332 €,Lufthansa
44,MIL-ROM,LIN Aeroporto Milano Linate,FCO Roma-Fiumicino,2021-10-06,10:20,19:15,1 scalo,340 €,Austrian Airlines
45,MIL-ROM,MXP Aeroporto Malpensa,FCO Roma-Fiumicino,2021-10-06,10:40,18:40,1 scalo,277 €,Turkish Airlines


In [92]:
train_details['arrival time'] = train_details['arrival time'].astype('string')
train_details['arrival time'] = train_details['arrival time'].str.rstrip(' +1').astype('string')
train_details['departure time'] = train_details['departure time'].astype('string')
train_details['departure time'] = train_details['departure time'].str.rstrip(' +1').astype('string')
train_details['date'] = train_details['date'].astype('string')
train_details['date'] = train_details['date'].str.rstrip(' +1').astype('string')
train_details.dtypes
train_details.tail()

,railway,departure station,arrival station,date,departure time,arrival time,change,price,type of train
41,MIL-ROM,LIN Aeroporto Milano Linate,FCO Roma-Fiumicino,2021-10-06,18:40,22:05,1 scalo,332 €,Lufthansa
42,MIL-ROM,MXP Aeroporto Malpensa,FCO Roma-Fiumicino,2021-10-06,13:30,17:15,1 scalo,332 €,Lufthansa
43,MIL-ROM,MXP Aeroporto Malpensa,FCO Roma-Fiumicino,2021-10-06,6:50,12:25,1 scalo,332 €,Lufthansa
44,MIL-ROM,LIN Aeroporto Milano Linate,FCO Roma-Fiumicino,2021-10-06,10:20,19:15,1 scalo,340 €,Austrian Airlines
45,MIL-ROM,MXP Aeroporto Malpensa,FCO Roma-Fiumicino,2021-10-06,10:40,18:40,1 scalo,277 €,Turkish Airlines


In [93]:
# train_details['departure time'] = pd.to_datetime(train_details['departure time'], errors = 'coerce',)
# train_details['arrival time'] = pd.to_datetime(train_details['arrival time'].astype('string'), errors = 'coerce')
# train_details.tail()
datetime_departure = train_details["date"] + " " + train_details["departure time"]
datetime_arrival = train_details["date"] + " " + train_details["arrival time"]
train_details['departure time'] = datetime_departure
train_details['arrival time'] = datetime_arrival
train_details['departure time'] = pd.to_datetime(train_details['departure time'], errors="coerce")
train_details['arrival time'] = pd.to_datetime(train_details['arrival time'], errors="coerce")
train_details.head()

,railway,departure station,arrival station,date,departure time,arrival time,change,price,type of train
0,MIL-ROM,MIL San Donato,ROM Tiburtina,2021-10-06,2021-10-06 23:15:00,2021-10-06 06:25:00,0 cambi,10 €,Itabus
1,MIL-ROM,MIL San Donato,ROM Tiburtina,2021-10-06,2021-10-06 23:15:00,2021-10-06 06:25:00,0 cambi,10 €,Itabus
2,MIL-ROM,MIL Centrale,ROM Tiburtina,2021-10-06,2021-10-06 14:48:00,2021-10-06 21:22:00,0 cambi,23 €,Trenitalia
3,MIL-ROM,MXP Aeroporto Malpensa,FCO Roma-Fiumicino,2021-10-06,2021-10-06 16:05:00,2021-10-06 21:35:00,1 scalo,16 €,Ryanair
4,MIL-ROM,MXP Aeroporto Malpensa,FCO Roma-Fiumicino,2021-10-06,2021-10-06 14:20:00,2021-10-06 18:45:00,1 scalo,17 €,"Wizz Air, Ryanair"


In [94]:
train_details.dtypes

railway                      object
departure station            object
arrival station              object
date                         string
departure time       datetime64[ns]
arrival time         datetime64[ns]
change                       object
price                        object
type of train                object
dtype: object

In [95]:
train_details['duration'] = train_details['arrival time'] - train_details['departure time']

In [96]:
train_details['departure time'] = pd.to_datetime(train_details['departure time'], errors = 'coerce').dt.time
train_details['arrival time'] = pd.to_datetime(train_details['arrival time'].astype('string'), errors = 'coerce').dt.time


In [97]:
train_details['duration'] = train_details['duration'].astype('string')
train_details['duration'] = train_details['duration'].str.split('s').str[-1].str.strip()
train_details['duration'] = train_details['duration'].str.split('\+').str[-1].str.strip()
train_details.tail()

,railway,departure station,arrival station,date,departure time,arrival time,change,price,type of train,duration
41,MIL-ROM,LIN Aeroporto Milano Linate,FCO Roma-Fiumicino,2021-10-06,18:40:00,22:05:00,1 scalo,332 €,Lufthansa,03:25:00
42,MIL-ROM,MXP Aeroporto Malpensa,FCO Roma-Fiumicino,2021-10-06,13:30:00,17:15:00,1 scalo,332 €,Lufthansa,03:45:00
43,MIL-ROM,MXP Aeroporto Malpensa,FCO Roma-Fiumicino,2021-10-06,06:50:00,12:25:00,1 scalo,332 €,Lufthansa,05:35:00
44,MIL-ROM,LIN Aeroporto Milano Linate,FCO Roma-Fiumicino,2021-10-06,10:20:00,19:15:00,1 scalo,340 €,Austrian Airlines,08:55:00
45,MIL-ROM,MXP Aeroporto Malpensa,FCO Roma-Fiumicino,2021-10-06,10:40:00,18:40:00,1 scalo,277 €,Turkish Airlines,08:00:00


In [98]:
train_details["price"].replace('', np.nan, inplace=True)
train_details = train_details.dropna(axis=0, how="any", subset=["price"])

In [99]:
train_details.rename(columns={'railway': 'airports', "departure station" : "departure airport", "arrival station" : "arrival airport", "type of train" : "airline"}, inplace=True)

In [100]:
# Get names of indexes for which column change has value cambi
index1 = train_details[(train_details['change'] == "1 cambio")].index
index2 = train_details[(train_details['change'] == "0 cambi")].index
index3 = train_details[(train_details['airline'].str.contains("FlixBus"))].index
# Delete these row indexes from dataFrame
train_details.drop(index=index1 , inplace=True)
train_details.drop(index=index2 , inplace=True)
train_details.drop(index=index3 , inplace=True)

In [101]:
train_details.to_csv("ds_flight_details_sc.csv")
train_details.to_excel("ds_flight_details_sc.xlsx")

In [103]:
x = pd.read_excel("/content/flights details.xlsx")
x.to_csv("flight details.csv")